<a href="https://colab.research.google.com/github/JohnGreco192/Free_GenAI_Bootcamp_2025/blob/main/GenAIBootCampweek1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ExamProCo/free-genai-bootcamp-2025.git

Cloning into 'free-genai-bootcamp-2025'...
remote: Enumerating objects: 457, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 457 (delta 17), reused 10 (delta 10), pack-reused 430 (from 1)
Receiving objects: 100% (457/457), 4.35 MiB | 21.84 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [5]:
import os

file_path = 'routes/study_sessions.py' # Corrected path to be relative to the current directory

# Get the content of the cell above this one (assuming the user will run this cell directly after the one with the code)
# WARNING: This method is fragile and assumes the user runs this cell immediately after the target code cell.
# A more robust method would involve reading the cell content programmatically if possible, or having the user copy-paste.
# For demonstration, we'll use a placeholder or assume the previous cell's content is available.
# In a real scenario with the Colab API, we would read the cell content by ID.
# Since we don't have direct access to the previous cell's *source code* in this interaction flow,
# I will generate the full content to be written based on the code I intended to put in that modify_cells command.

file_content = """
import os
from flask import request, jsonify, g
from flask_cors import cross_origin
from datetime import datetime
import math
import json # Import json to potentially handle request data

def load(app):
  # todo /study_sessions POST
  # Implementing the POST /study_sessions route
  @app.route('/study_sessions', methods=['POST']) # Added the intended route
  @cross_origin()
  def create_study_session():
    try:
      # Get group_id and study_activity_id from request JSON
      data = request.get_json()
      group_id = data.get('group_id')
      study_activity_id = data.get('study_activity_id')

      if not group_id or not study_activity_id:
        return jsonify({"error": "group_id and study_activity_id are required"}), 400

      cursor = app.db.cursor()

      # Check if group_id and study_activity_id exist (optional but good practice)
      cursor.execute("SELECT id FROM groups WHERE id = ?", (group_id,))
      group = cursor.fetchone()
      if not group:
          return jsonify({"error": f"Group with id {group_id} not found"}), 404

      cursor.execute("SELECT id FROM study_activities WHERE id = ?", (study_activity_id,))
      activity = cursor.fetchone()
      if not activity:
          return jsonify({"error": f"Study activity with id {study_activity_id} not found"}), 404

      # Insert the new study session
      cursor.execute('''
        INSERT INTO study_sessions (group_id, study_activity_id, created_at)
        VALUES (?, ?, ?)
      ''', (group_id, study_activity_id, datetime.utcnow().isoformat()))
      app.db.commit()

      # Get the ID of the newly created session
      session_id = cursor.lastrowid

      # For a new session, we might want to fetch words for this group
      # and potentially create initial word_review_items if needed for the activity type.
      # This part depends on the specific logic of study activities,
      # but we'll add a placeholder for fetching words.

      cursor.execute('''
          SELECT w.id, w.kanji, w.romaji, w.english
          FROM words w
          JOIN word_groups wg ON w.id = wg.word_id
          WHERE wg.group_id = ?
      ''', (group_id,))
      words_for_session = cursor.fetchall()


      return jsonify({
          "message": "Study session created successfully",
          "session_id": session_id,
          "group_id": group_id,
          "study_activity_id": study_activity_id,
          "words_in_session": [dict(word) for word in words_for_session] # Return words associated with the group
          }), 201

    except Exception as e:
      app.db.get().rollback() # Rollback changes if something goes wrong
      return jsonify({"error": str(e)}), 500


  @app.route('/api/study-sessions', methods=['GET'])
  @cross_origin()
  def get_study_sessions():
    try:
      cursor = app.db.cursor()

      # Get pagination parameters
      page = request.args.get('page', 1, type=int)
      per_page = request.args.get('per_page', 10, type=int)
      offset = (page - 1) * per_page

      # Get total count
      cursor.execute('''
        SELECT COUNT(*) as count
        FROM study_sessions ss
        JOIN groups g ON g.id = ss.group_id
        JOIN study_activities sa ON sa.id = ss.study_activity_id
      ''')
      total_count = cursor.fetchone()['count']

      # Get paginated sessions
      cursor.execute('''
        SELECT
          ss.id,
          ss.group_id,
          g.name as group_name,
          sa.id as activity_id,
          sa.name as activity_name,
          ss.created_at,
          COUNT(wri.id) as review_items_count
        FROM study_sessions ss
        JOIN groups g ON g.id = ss.group_id
        JOIN study_activities sa ON sa.id = ss.study_activity_id
        LEFT JOIN word_review_items wri ON wri.study_session_id = ss.id
        GROUP BY ss.id
        ORDER BY ss.created_at DESC
        LIMIT ? OFFSET ?
      ''', (per_page, offset))
      sessions = cursor.fetchall()

      return jsonify({
        'items': [{
          'id': session['id'],
          'group_id': session['group_id'],
          'group_name': session['group_name'],
          'activity_id': session['activity_id'],
          'activity_name': session['activity_name'],
          'start_time': session['created_at'],
          'end_time': session['created_at'],  # For now, just use the same time since we don't track end time
          'review_items_count': session['review_items_count']
        } for session in sessions],
        'total': total_count,
        'page': page,
        'per_page': per_page,
        'total_pages': math.ceil(total_count / per_page)
      })
    except Exception as e:
      return jsonify({"error": str(e)}), 500

  @app.route('/api/study-sessions/<id>', methods=['GET'])
  @cross_origin()
  def get_study_session(id):
    try:
      cursor = app.db.cursor()

      # Get session details
      cursor.execute('''
        SELECT
          ss.id,
          ss.group_id,
          g.name as group_name,
          sa.id as activity_id,
          sa.name as activity_name,
          ss.created_at,
          COUNT(wri.id) as review_items_count
        FROM study_sessions ss
        JOIN groups g ON g.id = ss.group_id
        JOIN study_activities sa ON sa.id = ss.study_activity_id
        LEFT JOIN word_review_items wri ON wri.study_session_id = ss.id
        WHERE ss.id = ?
        GROUP BY ss.id
      ''', (id,))

      session = cursor.fetchone()
      if not session:
        return jsonify({"error": "Study session not found"}), 404

      # Get pagination parameters
      page = request.args.get('page', 1, type=int)
      per_page = request.args.get('per_page', 10, type=int)
      offset = (page - 1) * per_page

      # Get the words reviewed in this session with their review status
      cursor.execute('''
        SELECT
          w.*,
          COALESCE(SUM(CASE WHEN wri.correct = 1 THEN 1 ELSE 0 END), 0) as session_correct_count,
          COALESCE(SUM(CASE WHEN wri.correct = 0 THEN 1 ELSE 0 END), 0) as session_wrong_count
        FROM words w
        JOIN word_review_items wri ON wri.word_id = w.id
        WHERE wri.study_session_id = ?
        GROUP BY w.id
        ORDER BY w.kanji
        LIMIT ? OFFSET ?
      ''', (id, per_page, offset))

      words = cursor.fetchall()

      # Get total count of words
      cursor.execute('''
        SELECT COUNT(DISTINCT w.id) as count
        FROM words w
        JOIN word_review_items wri ON wri.word_id = w.id
        WHERE wri.study_session_id = ?
      ''', (id,))

      total_count = cursor.fetchone()['count']

      return jsonify({
        'session': {
          'id': session['id'],
          'group_id': session['group_id'],
          'group_name': session['group_name'],
          'activity_id': session['activity_id'],
          'activity_name': session['activity_name'],
          'start_time': session['created_at'],
          'end_time': session['created_at'],  # For now, just use the same time
          'review_items_count': session['review_items_count']
        },
        'words': [{
          'id': word['id'],
          'kanji': word['kanji'],
          'romaji': word['romaji'],
          'english': word['english'],
          'correct_count': word['session_correct_count'],
          'wrong_count': word['session_wrong_count']
        } for word in words],
        'total': total_count,
        'page': page,
        'per_page': per_page,
        'total_pages': math.ceil(total_count / per_page)
      })
    except Exception as e:
      return jsonify({"error": str(e)}), 500

  # todo POST /study_sessions/:id/review
  # Implementing the POST /study_sessions/:id/review route
  @app.route('/study_sessions/<int:id>/review', methods=['POST']) # Added the intended route
  @cross_origin()
  def log_review_attempt(id):
      try:
          # Get word_id and correct status from request JSON
          data = request.get_json()
          word_id = data.get('word_id')
          correct = data.get('correct') # Expecting a boolean (True/False) or 1/0

          if word_id is None or correct is None:
              return jsonify({"error": "word_id and correct status are required"}), 400

          # Convert correct to integer (1 for True, 0 for False)
          correct_int = 1 if correct else 0

          cursor = app.db.cursor()

          # Check if the study session exists
          cursor.execute("SELECT id FROM study_sessions WHERE id = ?", (id,))
          session = cursor.fetchone()
          if not session:
              return jsonify({"error": f"Study session with id {id} not found"}), 404

          # Check if the word exists and is part of the session's group (optional validation)
          # For simplicity, we'll just check if the word exists
          cursor.execute("SELECT id FROM words WHERE id = ?", (word_id,))
          word = cursor.fetchone()
          if not word:
               return jsonify({"error": f"Word with id {word_id} not found"}), 404


          # Insert the word review item
          cursor.execute('''
              INSERT INTO word_review_items (study_session_id, word_id, correct, created_at)
              VALUES (?, ?, ?, ?)
          ''', (id, word_id, correct_int, datetime.utcnow().isoformat()))
          app.db.commit()

          # Update the word_reviews table (or insert if not exists)
          # This table aggregates review stats per word
          cursor.execute('''
              INSERT INTO word_reviews (word_id, correct_count, wrong_count)
              VALUES (?, ?, ?)
              ON CONFLICT(word_id) DO UPDATE SET
                  correct_count = correct_count + excluded.correct_count,
                  wrong_count = wrong_count + excluded.wrong_count;
          ''', (word_id, correct_int, 1 - correct_int)) # Add 1 to correct_count if correct, else to wrong_count
          app.db.commit()


          return jsonify({"message": "Review attempt logged successfully"}), 201

      except Exception as e:
          app.db.get().rollback() # Rollback changes if something goes wrong
          return jsonify({"error": str(e)}), 500


  @app.route('/api/study-sessions/reset', methods=['POST'])
  @cross_origin()
  def reset_study_sessions():
    try:
      cursor = app.db.cursor()

      # First delete all word review items since they have foreign key constraints
      cursor.execute('DELETE FROM word_review_items')

      # Then delete all study sessions
      cursor.execute('DELETE FROM study_sessions')

      # Also reset word_reviews table
      cursor.execute('DELETE FROM word_reviews')


      app.db.commit()

      return jsonify({"message": "Study history cleared successfully"}), 200
    except Exception as e:
      app.db.get().rollback() # Rollback changes if something goes wrong
      return jsonify({"error": str(e)}), 500
"""

try:
    with open(file_path, 'w') as f:
        f.write(file_content)
    print(f"Successfully wrote content to '{file_path}'")
except Exception as e:
    print(f"Error writing to file '{file_path}': {e}")

Successfully wrote content to 'routes/study_sessions.py'


In [16]:
# Ensure you are in the backend directory
%cd free-genai-bootcamp-2025/lang-portal/backend-flask

# Run the database initialization task
!invoke init-db

[Errno 2] No such file or directory: 'free-genai-bootcamp-2025/lang-portal/backend-flask'
/content/free-genai-bootcamp-2025/lang-portal/backend-flask
Successfully added 60 verbs to the 'Core Verbs' group.
Successfully added 64 verbs to the 'Core Adjectives' group.
Database initialized successfully.


In [17]:
# Install flask and ngrok
!pip install Flask Flask-Cors pyngrok

# Terminate any process using port 5000
!fuser -k 5000/tcp

# Navigate to the backend directory
%cd free-genai-bootcamp-2025/lang-portal/backend-flask

# Set the FLASK_APP environment variable
%env FLASK_APP=app.py

# Run the Flask application in a separate process and expose it with ngrok
import subprocess
from pyngrok import ngrok
import os
import time
from google.colab import userdata # Import userdata

# Terminate open tunnels if any
ngrok.kill()

# Get the authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("NGROK_AUTH_TOKEN successfully loaded from Colab secrets.")
else:
    print("NGROK_AUTH_TOKEN not found in Colab secrets.")
    print("Please add your authtoken to Colab secrets named 'NGROK_AUTH_TOKEN'.")


# Start Flask app using subprocess
# Use a simple command, disabling debug and reloader for potentially more stability
flask_process = subprocess.Popen(['flask', 'run', '--port', '5000', '--no-debugger', '--no-reload'])

# Give the Flask app a moment to start
time.sleep(5) # Increased sleep time

# Open a tunnel to the Flask app
try:
    public_url = ngrok.connect(5000)
    print(f"Flask app running at: {public_url}")
except Exception as e:
    print(f"Error starting ngrok tunnel: {e}")
    print("Attempting to terminate Flask process...")
    flask_process.terminate() # Attempt to stop the Flask process if ngrok fails
    print("Flask process terminated.")

# Note: The flask_process is running in the background.
# You might need to manually terminate it if you stop the cell execution
# or if it doesn't shut down cleanly when the notebook session ends.
# You can try !fuser -k 5000/tcp to kill it if needed.

5000/tcp:            60369
[Errno 2] No such file or directory: 'free-genai-bootcamp-2025/lang-portal/backend-flask'
/content/free-genai-bootcamp-2025/lang-portal/backend-flask
env: FLASK_APP=app.py
NGROK_AUTH_TOKEN successfully loaded from Colab secrets.
Flask app running at: NgrokTunnel: "https://152e-35-197-80-52.ngrok-free.app" -> "http://localhost:5000"


In [11]:
import os

backend_dir = '.' # Refer to the current directory

print(f"Contents of '{backend_dir}':\n")
# List files and directories in the backend directory to find setup scripts
for item in os.listdir(backend_dir):
    print(item)

print("\nContents of 'sql/':\n")
# List contents of the sql directory
sql_dir = os.path.join(backend_dir, 'sql')
if os.path.exists(sql_dir):
    for item in os.listdir(sql_dir):
        print(item)
else:
    print(f"Directory '{sql_dir}' not found.")

print("\nContents of 'seed/':\n")
# List contents of the seed directory
seed_dir = os.path.join(backend_dir, 'seed')
if os.path.exists(seed_dir):
    for item in os.listdir(seed_dir):
        print(item)
else:
    print(f"Directory '{seed_dir}' not found.")

Contents of '.':

tasks.py
seed
app.py
words.db
migrate.py
lib
__pycache__
.gitignore
routes
Readme.md
requirements.txt
sql

Contents of 'sql/':

setup

Contents of 'seed/':

study_activities.json
data_adjectives.json
data_verbs.json


In [15]:
import os

tasks_file_path = 'tasks.py' # Corrected path
migrate_file_path = 'migrate.py' # Corrected path
sql_setup_dir_path = 'sql/setup' # Corrected to directory path


print(f"Contents of '{tasks_file_path}':\n")
try:
    with open(tasks_file_path, 'r') as f:
        print(f.read())
except FileNotFoundError:
    print(f"'{tasks_file_path}' not found.")

print(f"\nContents of '{migrate_file_path}':\n")
try:
    with open(migrate_file_path, 'r') as f:
        print(f.read())
except FileNotFoundError:
    print(f"'{migrate_file_path}' not found.")

print(f"\nContents of directory '{sql_setup_dir_path}':\n")
try:
    if os.path.isdir(sql_setup_dir_path):
        for item in os.listdir(sql_setup_dir_path):
            print(item)
    else:
        print(f"'{sql_setup_dir_path}' is not a directory or does not exist.")
except FileNotFoundError:
    print(f"Directory '{sql_setup_dir_path}' not found.")

Contents of 'tasks.py':

from invoke import task
from lib.db import db

@task
def init_db(c):
  from flask import Flask
  app = Flask(__name__)
  db.init(app)
  print("Database initialized successfully.")

Contents of 'migrate.py':

import sqlite3
import os

def run_migrations():
    # Connect to the database
    db_path = os.path.join(os.path.dirname(__file__), 'word_bank.db')
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row
    
    try:
        # Get list of migration files
        migrations_dir = os.path.join(os.path.dirname(__file__), 'sql', 'migrations')
        migration_files = sorted([f for f in os.listdir(migrations_dir) if f.endswith('.sql')])
        
        # Run each migration
        for migration_file in migration_files:
            print(f"Running migration: {migration_file}")
            with open(os.path.join(migrations_dir, migration_file)) as f:
                migration_sql = f.read()
                conn.executescript(migration_sql)
      